# Table of contents
- [01 Importing Moduls](#01-Importing-Moduls)

### 01 Importing Moduls

In [1]:
#!pip install yfinance 


In [3]:
import numpy as np
import pandas as pd 
import yfinance as yf
import datetime
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('bmh')
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
# in this case: Zoom

data=yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30",threads= False)
data.info()

In [3]:
data['datetime']=data.index
data.reset_index()
#data.drop("Date", inplace=True)

NameError: name 'data' is not defined

In [4]:
data.head()

NameError: name 'data' is not defined

In [ ]:
#!pip install ta # for getting financial indicators 
from ta import add_all_ta_features 


In [ ]:
data = add_all_ta_features(
    data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=[15,8])
plt.plot(data.datetime,data.Open)
plt.xlabel("Date")
plt.ylabel("Zoom Opening Stock")

In [ ]:
#pip install finnhub-python
#!pip install pystan==2.19.1.1.
#!pip install  prophet

In [ ]:
import finnhub

# Setup client
finnhub_client = finnhub.Client(api_key="c2si65iad3ic1qis06lg")

In [ ]:
def get_news(company, date_from='2020-01-01', date_to=None):
    '''
    returns dataframe with average sentiment of news headline and sentiment of news summary for every date in a given timeframe
    company: symbol, example ZM
    date_from: string format yyyy-mm-dd
    date_to: string format yyyy-mm-dd
    '''
    sid_obj = sia()
    if date_to is None:
        date_to = datetime.date.today().strftime("%Y-%m-%d")
    result=(finnhub_client.company_news(company, _from=date_from, to=date_to))
    news_df = pd.DataFrame(result)
    news_df['datetime'] = [datetime.datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in news_df.datetime]
    news_df.drop(['id','image', 'related','source', 'url'], axis=1, inplace=True)
    news_df['headline_sentiment'] = [sid_obj.polarity_scores(c)['compound'] for c in news_df['headline']]
    news_df['summary_sentiment'] = [sid_obj.polarity_scores(c)['compound'] for c in news_df['summary']]
    news_dates = news_df.groupby(['datetime']).mean().sort_index().reset_index()
    return news_dates

In [ ]:
news_dates=get_news("ZM", date_from='2020-01-01')
#news_dates.info()

In [ ]:
news_dates['datetime']=news_dates['datetime'].astype('datetime64[ns]')#datetime64
data['datetime']=data['datetime'].astype('datetime64[ns]')
df=news_dates.merge(data,on="datetime",how='inner')

In [ ]:
news_dates.tail()

In [ ]:
df_plot=df[df.datetime > '2021-03-01']

In [ ]:
plt.figure(figsize=[30,8])
x1 = df_plot.datetime
y1 = df_plot.Open
# plotting the line 1 points 
plt.plot(x1, y1, label = "line 1")
# line 2 points
x2 = df_plot.datetime
y2 = df_plot.headline_sentiment*100
# plotting the line 2 points 
plt.plot(x2, y2, label = "line 2")
#plt.yscale("log")
plt.xlabel('x - axis')
# Set the y axis label of the current axis.
plt.ylabel('y - axis')
# Set a title of the current axes.
plt.title('Two or more lines on same plot with suitable legends ')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
df

# Random Forest for Feature Importance 

In [ ]:
#shifting 
move_days = 7
# shifted values column, like this the model can learn what the price is going to be x days later
df["shift_close"]=df[["Close"]].shift(-move_days)

In [ ]:
df.tail(10)

In [ ]:
#x without the NaNs 
X = np.array(df.drop(['shift_close','datetime'],1))[:-move_days]
X = np.array(df.drop(['shift_close','datetime'],1))[:-move_days]
y = df.shift_close[:-move_days]

In [ ]:
# have to make this work somehow
#X= SelectKBest(f_regression, k=20).fit_transform(X, y)
#X = X[:-move_days]

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
# training a model on the dataset where we have all data
param_grid = {'min_samples_leaf': [2,3,5,10,20], 'n_estimators': [3,5,10,50,100]}
rf = RandomForestRegressor(random_state = 42)
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 10)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
rf=RandomForestRegressor(min_samples_leaf=2, n_estimators=3, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred_existing=rf.predict(X_test)

In [ ]:
print(mean_squared_error(y_test, y_pred_existing))

In [ ]:
# Now the X variable is going to be the features for the days where we have no "in x Days price"
X_pred=df.drop(['shift_close','datetime'],1)[:-move_days]
X_pred=X_pred.tail(move_days)
#X_pred=np.array(X_pred)
#print(X_pred)

In [ ]:
#values for the next 7 days 
pred=rf.predict(X_pred)
pred

In [ ]:
df_pred=pd.DataFrame(pred)

In [ ]:
pd.to_datetime(df.datetime)

In [ ]:
df_pred

In [ ]:
date_pred=[]
for i in range (1,move_days):
    date_pred.append(df.datetime.iloc[-1]+timedelta(days=i))

In [ ]:
date_pred.todatetime

In [ ]:
df.info()

In [ ]:
from datetime import timedelta, date

In [ ]:
for i in range(1,move_days):
    df_pred['pred_date']=df.datetime.iloc[-1]+timedelta(days=i)
    df_pred.pred_date.astype('datetime64[ns]')

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_pred)


In [ ]:
df.columns

In [ ]:
shap.summary_plot(shap_values, X_test,feature_names=X_test.columns,max_display=30)

In [ ]:
df_feature_importance = pd.DataFrame(rf.feature_importances_, index=X_pred.columns, columns=['feature importance']).sort_values('feature importance', ascending=False)
print(df_feature_importance)

In [ ]:
plt.plot(np.arange(1,move_days+1,1),pred)
plt.xlabel("Days from now")
plt.ylabel("Price")

### Need to think of a way to plot this nicely

## Facebook Prophet on Close Price

In [ ]:
# FB Prophet 

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly

In [ ]:
df

In [ ]:
data.reset_index(inplace=True)
df_train = data[['datetime','Close']]
df_train = df_train.rename(columns={"datetime": "ds", "Close": "y"})
df_train

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(df_train)
future = m.make_future_dataframe(periods=21, freq ='D')
forecast = m.predict(future)

In [ ]:
forecast.tail()

In [ ]:
m.plot(forecast)
plt.show()

## LSTM Model (code should be correct, but somethings wrong with the package) 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [ ]:
len(data)/2

In [ ]:
new_data = data[['datetime','Close']]
for i in range(0,len(data)):
    new_data['datetime'][i] = data['datetime'][i]
    new_data['Close'][i] = data['Close'][i]

#setting index
new_data.index = new_data.datetime
new_data.drop('datetime', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:274,:]
valid = dataset[274:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)